In [9]:
import requests
from bs4 import BeautifulSoup
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
from urllib.parse import urlparse
import re
import time
from pprint import pprint 
from tqdm import tqdm
from typing import Optional, Callable

In [3]:
from config import (
    ELASTICSEARCH_URL,
    INDEX_NAME_HUGGINGFACE,
    INDEX_NAME_GEMINI,
    INDEX_NAME_OPENAI,
    SITEMAP_URL,
    HUGGINGFACE_EMBEDDING_MODEL_NAME,
    GEMINI_EMBEDDING_MODEL_NAME,
    OPENAI_EMBEDDING_MODEL_NAME
)

**Connect To Elastic Search**

In [ ]:
es = Elasticsearch(ELASTICSEARCH_URL)
print("Connected to Elasticsearch!")

client_info = es.info()

Crate Index utils function

In [ ]:
def create_index(index_name: str):
    if es.indices.exists(index=index_name):
        print(f"Index '{index_name}' already exists.")
        return

    mapping = {
        "mappings": {
            "properties": {
                "url": {"type": "keyword"},
                "title": {"type": "text"},
                "content": {"type": "text"},
                "embedding": {"type": "dense_vector", "dims": 384}
            }
        }
    }

    es.indices.create(index=index_name, body=mapping)
    print(f"Index '{index_name}' created successfully.")

Extract page content utils function

In [ ]:
def extract_page_text(url):
    try:
        res = requests.get(url, timeout=10)
        res.raise_for_status()
        soup = BeautifulSoup(res.text, "html.parser")

        # Remove unwanted elements
        for tag in soup(["script", "style", "noscript"]):
            tag.extract()

        title = soup.title.string.strip() if soup.title else "Untitled"
        text = re.sub(r"\s+", " ", soup.get_text(separator=" ", strip=True))
        return title, text

    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return None, None

get_embedder model utils function

In [ ]:
def get_embedder(model_type: str, api_key: Optional[str] = None) -> Callable[[str], list[float]]:
    """
    Initializes the embedding model and returns a function to encode text.
    
    Args:
        model_type: 'HuggingFace', 'Gemini', or 'OpenAI'.
        api_key: The necessary API key for the chosen service (if applicable).
        
    Returns:
        A callable function that takes a text string and returns a list of floats (embedding).
    """
    model_type = model_type.lower()

    if model_type == "huggingface":
        # Requires: pip install sentence-transformers
        # The specific model (e.g., 'all-MiniLM-L6-v2') can be passed as an argument
        try:
            from sentence_transformers import SentenceTransformer
        except ImportError:
            raise ImportError("HuggingFace embedder requires 'sentence-transformers'. Please install it.")
        
        # Initialize the model (e.g., a common sentence-transformer model)
        hf_model = SentenceTransformer('all-MiniLM-L6-v2') 
        # Return a lambda that calls the encode method and converts to list
        return lambda content: hf_model.encode(content).tolist()

    elif model_type == "gemini":
        # Requires: pip install google-genai
        try:
            from google import genai
        except ImportError:
            raise ImportError("Gemini embedder requires 'google-genai'. Please install it.")
        
        if not api_key:
             raise ValueError("Gemini embedder requires an API key.")

        client = genai.Client(api_key=api_key)
        
        def gemini_embed(content: str) -> list[float]:
            # Use the 'embedding-001' model for general embeddings
            response = client.models.embed_content(
                model='models/embedding-001',
                content=content,
                task_type='RETRIEVAL_DOCUMENT'
            )
            return response['embedding']

        return gemini_embed

    elif model_type == "openai":
        # Requires: pip install openai
        try:
            from openai import OpenAI
        except ImportError:
            raise ImportError("OpenAI embedder requires 'openai'. Please install it.")
        
        if not api_key:
             raise ValueError("OpenAI embedder requires an API key.")
             
        client = OpenAI(api_key=api_key)

        def openai_embed(content: str) -> list[float]:
            # Use the standard text-embedding-3-small model
            response = client.embeddings.create(
                input=content,
                model="text-embedding-3-small"
            )
            # The API returns a list of embeddings, we take the first (and only) one
            return response.data[0].embedding

        return openai_embed

    else:
        raise ValueError(f"Unknown model type: {model_type}. Must be 'HuggingFace', 'Gemini', or 'OpenAI'.")

scrape sitemap and index pages utils func

In [ ]:
def scrape_and_index(model_type: str, index_name: str, api_key: Optional[str] = None):
    # 1. Get Embedder Function
    try:
        # Get the callable function that performs the embedding
        embed_content = get_embedder(model_type, api_key=api_key) 
    except (ImportError, ValueError) as e:
        print(f"FATAL ERROR: Could not initialize embedder. {e}")
        return

    # 2. Scrape Sitemap
    print(f"Fetching sitemap from: {SITEMAP_URL}")
    sitemap_xml = requests.get(SITEMAP_URL).text
    soup = BeautifulSoup(sitemap_xml, "xml")

    # Filter URLs to the specific path
    urls = [loc.text for loc in soup.find_all("loc") if "/en-ie/" in loc.text]
    
    print(f"Found **{len(urls)}** URLs to crawl.")
    pprint(urls[:5])
    print("...")

    # 3. Process and Index Pages
    total_urls = len(urls)
    operations = []
    count_operation = 0
    batch_size = 50
    for url in tqdm(urls, total=total_urls, desc="Fetching and Indexing pages..."):
        count_operation += 1
        operations.append({"index": {"_index": index_name}})
        try:
            # Scrape content
            title, content = extract_page_text(url)
            
            # Skip if content is empty
            if not content:
                print(f"⏭Skipped ({i}/{total_urls}): Empty content for {url}")
                continue

            # Generate Embedding
            # Now we use the returned callable function 'embed_content'
            embedding = embed_content(content) 

            # Create Document
            doc = {
                "url": url,
                "title": title,
                "content": content,
                "embedding": embedding
            }

            operations.append(doc)

            if count_operation >= batch_size:
                es.bulk(operations=operations)
                count_operation = 0
                operations.clear()

            # Index Document
            es.index(index=index_name, document=doc)
            print(f"Indexed: {url}")

        except Exception as e:
            print(f"Failed to process/index {url}. Error: {e}")
    
    if operations:
        es.bulk(operations=operations)


Found 141 URLs to crawl.
['https://www.sisuclinic.com/en-ie/treatments',
 'https://www.sisuclinic.com/en-ie/providers',
 'https://www.sisuclinic.com/en-ie/faqs',
 'https://www.sisuclinic.com/en-ie/locations',
 'https://www.sisuclinic.com/en-ie/about-us',
 'https://www.sisuclinic.com/en-ie/pricing',
 'https://www.sisuclinic.com/en-ie/results',
 'https://www.sisuclinic.com/en-ie/accessibility',
 'https://www.sisuclinic.com/en-ie/contact-us',
 'https://www.sisuclinic.com/en-ie/get-the-look']
...


**Scrape the pages from the sitemap and use HuggingFace Embedding Model**

In [12]:
my_list = [5, 4, 3, 2, 1]
for i, num in enumerate(my_list):
    print(i, num)

0 5
1 4
2 3
3 2
4 1
